### 1. MongoDB에 웹스크래핑 데이터 저장하기
* MongoDB에 연결
* database와 collection 생성
* actors.json 읽어서 insert_many()

In [8]:
import pymongo
import json

con = pymongo.MongoClient(host='localhost',port=27017)
print(con)

db = con.member_db
print(db)

#collection drop 
db.drop_collection('actors')

col = db.actors
print(col)

with open('data/actors.json','r',encoding='utf8') as file:
    json_data = json.load(file)
    col.insert_many(json_data)

print('document 건수', col.estimated_document_count())    

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'member_db')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'member_db'), 'actors')
document 건수 44


In [9]:
#컬렉션의 find()를 사용한 함수 정의
def print_find_document(criteria, projection):
    for idx, member_doc in enumerate(col.find(criteria, projection),1):
        print(idx, member_doc)

In [10]:
#컬렉션의 aggregate()를 사용한 함수 정의
def print_aggregate_document(*args):
    agg_list = []
    for arg in args:
        agg_list.append(arg)
    print(agg_list)
    
    for agg_doc in col.aggregate(agg_list):
        print(agg_doc)

In [11]:
#1. 랭킹 1 ~ 50 인 배우의 배우이름,랭킹,흥행지수,소속사,출연영화
criteria = {
    '랭킹':{'$lte':50}
}
projection = {
    '_id':0,'랭킹':1,'배우이름':1,'흥행지수':1,'소속사':1,'출연영화':1
}

print_find_document(criteria, projection)

1 {'배우이름': '이병헌', '랭킹': 1, '흥행지수': '21472', '출연영화': ['비상선언', '콘크리트 유토피아', '백두산']}
2 {'배우이름': '진선규', '랭킹': 2, '흥행지수': '19105', '출연영화': ['카운트(가제)', '킹덤 시즌 2', '사바하']}
3 {'배우이름': '하정우', '랭킹': 3, '흥행지수': '17579', '출연영화': ['클로젯', '백두산', 'PMC: 더 벙커']}
4 {'배우이름': '마동석', '랭킹': 4, '흥행지수': '17337', '출연영화': ['시동', '나쁜 녀석들: 더 무비', '백두산']}
5 {'배우이름': '이하늬', '랭킹': 5, '흥행지수': '15912', '출연영화': ['블랙머니', '열혈사제', '극한직업']}
6 {'배우이름': '류승룡', '랭킹': 6, '흥행지수': '14030', '소속사': '필름있수다', '출연영화': ['휴머니멀', '킹덤 시즌 2', '입술은 안돼요(가제)']}
7 {'배우이름': '유해진', '랭킹': 7, '흥행지수': '13690', '소속사': '심엔터테인먼트', '출연영화': ['휴머니멀', '삼시세끼 어촌편 5', '봉오동 전투']}
8 {'배우이름': '박정민', '랭킹': 8, '흥행지수': '12976', '출연영화': ['사냥의 시간', '다만 악에서 구하소서', '사바하']}
9 {'배우이름': '황정민', '랭킹': 9, '흥행지수': '12863', '소속사': '예당엔터테인먼트', '출연영화': ['다만 악에서 구하소서', '교섭', '공작']}
10 {'배우이름': '송강호', '랭킹': 10, '흥행지수': '12429', '소속사': '호두엔터테인먼트', '출연영화': ['비상선언', '브로커(가제)', '기생충']}
11 {'배우이름': '조진웅', '랭킹': 11, '흥행지수': '11848', '소속사': '㈜사람엔터테인먼트', '출연영화': ['경관의 피(가제)', '대외비: 권력의 

In [23]:
for actor in col.find({'흥행지수':{'$gte':20000}},{'_id':0,'배우이름':1,'흥행지수':1})\
.sort('배우이름',pymongo.DESCENDING):
    print(actor)

In [19]:
##$exists 연산자를 사용
#특기 필드가 존재하지 않ㄴ는 ㅇ경우와 존재하는 경ㅇ우 count
print(col.find({'특기':{'$exist':True}}))
print(col.find({'특기':{'$exist':False}}))

In [21]:
criteria={'다른 이름':{'$exists':True},'다른 이름':{'$regex':'방탄소년단'}}

projection={'출연영화':{'$exists':True},'다른 이름':{'$regex':'방탄소년단'}}



In [25]:
#$nor연산자 not or 

print(col.count_documents({'$or':[{'흥행지수':{'$gte':15000}},{'성별':'남'}]}))
print(col.count_documents({'$nor':[{'흥행지수':{'$lte':15000}},{'성별':'여'}]}))

32
32


In [31]:
criteria={'$nor':[{'흥행지수':{'$lte':15000}},{'성별':'여'}]}
print_find_document(criteria,projection)

1 {'배우이름': '이병헌', '랭킹': 1, '흥행지수': '21472', '출연영화': ['비상선언', '콘크리트 유토피아', '백두산']}
2 {'배우이름': '진선규', '랭킹': 2, '흥행지수': '19105', '출연영화': ['카운트(가제)', '킹덤 시즌 2', '사바하']}
3 {'배우이름': '하정우', '랭킹': 3, '흥행지수': '17579', '출연영화': ['클로젯', '백두산', 'PMC: 더 벙커']}
4 {'배우이름': '마동석', '랭킹': 4, '흥행지수': '17337', '출연영화': ['시동', '나쁜 녀석들: 더 무비', '백두산']}
5 {'배우이름': '류승룡', '랭킹': 6, '흥행지수': '14030', '소속사': '필름있수다', '출연영화': ['휴머니멀', '킹덤 시즌 2', '입술은 안돼요(가제)']}
6 {'배우이름': '유해진', '랭킹': 7, '흥행지수': '13690', '소속사': '심엔터테인먼트', '출연영화': ['휴머니멀', '삼시세끼 어촌편 5', '봉오동 전투']}
7 {'배우이름': '박정민', '랭킹': 8, '흥행지수': '12976', '출연영화': ['사냥의 시간', '다만 악에서 구하소서', '사바하']}
8 {'배우이름': '황정민', '랭킹': 9, '흥행지수': '12863', '소속사': '예당엔터테인먼트', '출연영화': ['다만 악에서 구하소서', '교섭', '공작']}
9 {'배우이름': '송강호', '랭킹': 10, '흥행지수': '12429', '소속사': '호두엔터테인먼트', '출연영화': ['비상선언', '브로커(가제)', '기생충']}
10 {'배우이름': '조진웅', '랭킹': 11, '흥행지수': '11848', '소속사': '㈜사람엔터테인먼트', '출연영화': ['경관의 피(가제)', '대외비: 권력의 탄생(가제)', '광대들: 풍문조작단']}
11 {'배우이름': '신하균', '랭킹': 12, '흥행지수': '11173', '출연영화': [

### list 에 포함된 문자열 검색
*$alll 연산자에 사용하면 순서에 상관없이 값으로 해당 문자열 찾을수있씀

In [36]:
col.count_documents({'출연영화':'아무도 모른다'})
criteria={'출연영화':{'$all':['아무도 모른다', '특별근로감독관 조장풍']}}
print_find_document(criteria,projection)

In [37]:
sort={'$sortByCount':'$소속사'}

print_aggregate_document(sort)

[{'$sortByCount': '$소속사'}]
{'_id': None, 'count': 27}
{'_id': '아티스트컴퍼니', 'count': 5}
{'_id': '호두엔터테인먼트', 'count': 3}
{'_id': '예당엔터테인먼트', 'count': 2}
{'_id': '심엔터테인먼트', 'count': 2}
{'_id': 'SM엔터테인먼트', 'count': 1}
{'_id': '㈜사람엔터테인먼트', 'count': 1}
{'_id': 'JYP엔터테인먼트', 'count': 1}
{'_id': '필름있수다', 'count': 1}
{'_id': '스타파크엔터테인먼트', 'count': 1}
